# Finding names in book reviews

In [1]:
#import necessary packages
import os
import pandas as pd
import re
from collections import Counter
import numpy as np
%pprint

Pretty printing has been turned OFF


In [2]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [3]:
import pickle

In [4]:
df_all = pickle.load(open('author_title_df.pkl', 'rb'))

In [5]:
authors_by_last_name = pickle.load(open('author_last_name_dict.pkl', 'rb'))

### Step 1: Find names using titles and capitalization

In [6]:
titles = """Doctor,Dr,Mr,Mrs,Ms,Miss,Msgr,Monsignor,Rev,Reverend,Hon,Honorable,Honourable,Prof,Professor,Madame,Madam,Lady,Lord,Sir,Dame,Master,Mistress,Chancellor,Principal,President,Pres,Warden,Dean,Regent,Rector,Provost,Director
"""

In [7]:
titles = titles.rstrip().split(',')

In [8]:
'Doctor' in titles

True

In [9]:
title_list = '.?\s|'.join(titles)

In [10]:
# test text
mytext = """
    Missouri: A Bone of Contention '                 WE  aware that most of the States of the Union had their nicknames, more or less complimentary, but to name Missouri ' a bone of contention ' is a stroke of wit. It does, however, rightly describe the Missouri of the past, and vividly writes in a phrase her political history. Until the triumph of the Union armies and the close of the Civil War, Missouri was in the jaws of the watch-dogs of slavery and freedom. In war or in peace, the subiect of legislative com-                 promise or of military struggle, Missouri was an uncertain factor. Now, after -five years of national peace, her history may be calmly and impartially written. Indeed, the task has been done, and well done, and the , Lucien Carr of Harvard, may be congratulated upon his work,  is strong, unimpassioned, scholarly, and as impressed with the firm touch which comes of local knowledge as are the imprinted rocks in the cabinets at Cambridge. Long familiarity with the wealth of archaeology in the Peabody Museuml seems to have given him the power of comparison and generalization in the evolution of a commonwealth, while  acquaintance with living men enables him to blend the results of the study and the field in pleasing literary form. Five of his seventeen chapters give a luminous picture of the early French and Spanish discoveries and domination. Then follow three chapters treating of the                 Missouri. By Ldclen Carr. $ti... (American Commonwealths.) Boston:                 Itoughto., Sftfltn & Co.                 l , the compromise, and the  into the U nion of this State named after the great river which flows through it. In his treatment of the period from 1844 to i861, as well as that of war time, some readers may charge Mr. Carr with unduly favoring the Southern and even Confederate view; but to people living this side of the now-vanished Mason and Dixon's line, this is doubtless a benefit; for only when Northern people are able to ' put themselves in the place' of Southerners and see with Southern eyes, can they be sure that they have achieved that impartiality which is essential to the writing of final history. He shows that the Missourians were neither secessionists nor slavery propagandists. He both criticises and justifies the action of the second convention which, in the uncertain hours when other States  seceding and Missouri's Governor had been driven into exile, org  a provisional government, and  saved Missouri Irons ' the pit of political degradation into which the States in rebellion were sunk during the period of reconstruction.' Mr. Carr practically and almost abruptly ends his history at the close of the War, believing that the career of Missouri as a bone of contention ended with the abolition of slavery. The fifty years' struggle was over, the State recovered rapidly froni the wounds of the Civil War, wealth increased wonderfully, and the Negro was liberally dealt with in most if not all points relating to citizenship. Taken as a whole, this book, with its sustained interest, high average literary merit, and thorough treatment of the voluminous facts, fully justifies its place in the series of ' histories of such States as have exercised a positive influence in the shaping of the national Government, or have had a striking political - . . history.' Like the others, it has a good map and index."""  

In [11]:
mytext.split()

['Missouri:', 'A', 'Bone', 'of', 'Contention', "'", 'WE', 'aware', 'that', 'most', 'of', 'the', 'States', 'of', 'the', 'Union', 'had', 'their', 'nicknames,', 'more', 'or', 'less', 'complimentary,', 'but', 'to', 'name', 'Missouri', "'", 'a', 'bone', 'of', 'contention', "'", 'is', 'a', 'stroke', 'of', 'wit.', 'It', 'does,', 'however,', 'rightly', 'describe', 'the', 'Missouri', 'of', 'the', 'past,', 'and', 'vividly', 'writes', 'in', 'a', 'phrase', 'her', 'political', 'history.', 'Until', 'the', 'triumph', 'of', 'the', 'Union', 'armies', 'and', 'the', 'close', 'of', 'the', 'Civil', 'War,', 'Missouri', 'was', 'in', 'the', 'jaws', 'of', 'the', 'watch-dogs', 'of', 'slavery', 'and', 'freedom.', 'In', 'war', 'or', 'in', 'peace,', 'the', 'subiect', 'of', 'legislative', 'com-', 'promise', 'or', 'of', 'military', 'struggle,', 'Missouri', 'was', 'an', 'uncertain', 'factor.', 'Now,', 'after', '-five', 'years', 'of', 'national', 'peace,', 'her', 'history', 'may', 'be', 'calmly', 'and', 'impartially',

In [12]:
from symspellpy.symspellpy import SymSpell, Verbosity 

# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
first_name_symspell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
dictionary_path = "name_freq_dict.txt"
term_index = 0 
count_index = 1
first_name_symspell.load_dictionary(dictionary_path, term_index, count_index, separator=",")

True

In [13]:
authors_last_name_list = ','.join(list(authors_by_last_name.keys()))

In [14]:
f = open("authors_last_name_list.txt", "w")
f.write(authors_last_name_list)
f.close()

In [15]:
# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
author_surname_symspell = SymSpell()
# load dictionary
term_index = 0 
count_index = 1
author_surname_symspell.create_dictionary('authors_last_name_list.txt')

True

In [16]:
author_surname_symspell._words['carr']

2

In [17]:
author_surname_symspell.lookup('carm', Verbosity.CLOSEST)[0]._term

'carr'

In [18]:
from nltk.corpus import stopwords

In [19]:
stopword_list = stopwords.words('english')

In [20]:
'by' in stopword_list

True

In [21]:
re.findall(title_list, mytext)

['Mr. ', 'Mr. ']

In [22]:
iter = re.finditer(title_list, mytext)

In [23]:
indices = [(m.start(), m.group()) for m in iter]
# indices = [m.start(0) for m in iter]

In [24]:
indices[0][1]

'Mr. '

In [25]:
for i, index in enumerate(indices):
    print(i, index)

0 (1785, 'Mr. ')
1 (2624, 'Mr. ')


In [26]:
re.match('Mr.', mytext[1785:]).group()

'Mr.'

In [27]:
re.match(indices[0][1] + '.*? [^A-Z]', mytext[indices[0][0]:])

<re.Match object; span=(0, 10), match='Mr. Carr w'>

In [28]:
ex_txt = 'Here is an example text that features a longer name. This name is Mr James Egg-Briggs and he is an author.'

In [29]:
for i, index in enumerate(indices):
    if i < len(indices)-1:
        print(index)

(1785, 'Mr. ')


In [30]:
iterex = re.finditer(title_list, txts[0])

In [31]:
indices = [(m.start(), m.group()) for m in iterex]

In [32]:
indices

[(218, 'Mr. '), (325, 'Mr. '), (556, 'Professor '), (848, 'Mrs. '), (1020, 'Mr. '), (1052, 'Dr. '), (1631, 'Dr. '), (1927, 'Mr. '), (2086, 'Mr. '), (2706, 'Dr. '), (4147, 'Mr. '), (4460, 'Dr. '), (4599, 'President '), (5450, 'Mr. '), (5737, 'Dr. '), (5806, 'Professor '), (5917, 'Dr. '), (5930, 'Dr. ')]

In [41]:
for i, index in enumerate(indices):
    if (i<len(indices)-1):
        print(re.match(indices[i][1] + '.*? [^A-Z]', txts[0][indices[i][0]:indices[i+1][0]]).group()[:-2])

Mr. Charles Dudley Warner
Mr. Vedder's
Professor Thorold Rogers
Mrs. Browning
Mr. Swinburne.
Dr. William
Dr. Geor6 Ebers'
Mr. Julian Hawthorne
Mr. Julius Chambers,
Dr. 
Mr. Cushing
Dr. Herbert B. Adam.
President Andrew D. White.
Mr. Brander Matthews,
Dr. Joseph Parktercontinueshiseipositionuin
Professor E. Johnsou


AttributeError: 'NoneType' object has no attribute 'group'

In [38]:
def getNamesFollowingTitles(txt):
    """
    Returns names following titles - specifically capitalized titles followed by capitalized names.
    Names can be any number of words in length, and can include punctuation.
    """
    names = []
    
    # iter only works once before emptying
    iterx = re.finditer(title_list, txt)
    indices = [(m.start(), m.group()) for m in iterx]
    
    for i, index in enumerate(indices):
        if i < len(indices)-1:
            names.append(re.match(indices[i][1] + '.*? [^A-Z]|/Z', txt[indices[i][0]:indices[i+1][0]]).group()[:-2])
        else:
            names.append(re.match(indices[i][1] + '.*? [^A-Z]|/Z', txt[indices[i][0]:]).group()[:-2])
    
    return list(set(names))

In [39]:
getNamesFollowingTitles(txts[0])

AttributeError: 'NoneType' object has no attribute 'group'

In [43]:
# potential one word names
one_words = [match for match in re.findall('[A-Z]\S+', mytext) if (removePunct(match).lower() not in stopword_list) 
             and (removePunct(match).lower() not in mytext)]

In [44]:
one_words

['Missouri:', 'States', 'Union', 'Missouri', 'Missouri', 'Union', 'Civil', 'Missouri', 'Missouri', 'Indeed,', 'Lucien', 'Carr', 'Harvard,', 'Cambridge.', 'Long', 'Peabody', 'Museuml', 'French', 'Spanish', 'Missouri.', 'Ldclen', 'Carr.', 'American', 'Commonwealths.)', 'Boston:', 'Itoughto.,', 'Sftfltn', 'State', 'Mr.', 'Carr', 'Southern', 'Confederate', 'Mason', "Dixon's", 'Northern', 'Southerners', 'Southern', 'Missourians', 'States', "Missouri's", 'Governor', 'Missouri', 'Irons', 'States', 'Mr.', 'Carr', 'Missouri', 'State', 'Civil', 'Negro', 'Taken', 'States', 'Like']

In [28]:
# potential two word and three word names
two_words = re.findall('[A-Z]\S* [A-Z]\S+', mytext)
three_words = re.findall('[A-Z]\S* [A-Z]\S* [A-Z]\S+', mytext)

In [29]:
print(two_words)
print(three_words)

['A Bone', 'Civil War,', 'Lucien Carr', 'Cambridge. Long', 'Peabody Museuml', 'Missouri. By', 'Ldclen Carr.', 'American Commonwealths.)', 'Itoughto., Sftfltn', 'Mr. Carr', "Missouri's Governor", 'Missouri Irons', 'Mr. Carr', 'Civil War,']
['Missouri: A Bone', 'Civil War, Missouri', 'Missouri. By Ldclen', 'American Commonwealths.) Boston:']


In [34]:
def removePunct(word):
    return ''.join([x for x in word if (x.isalpha()) or (x == '-') or (x=='–')])

In [35]:
removePunct('Egg-Briggs.')

'Egg-Briggs'

In [80]:
re.search('([A-Z]\S* )*[A-Z]\S*', mytext)

<re.Match object; span=(5, 21), match='Missouri: A Bone'>

In [95]:
ex = 'Missouri'

In [94]:
ex_list = ["Missouri's Governor","Egg"]

In [98]:
[ex in x for x in ex_list]

[True, False]

In [231]:
def getCapitalizedWords(txt):
    """
    Returns strings of capitalized words up 3 words long. 
    Removes words/phrases containing stopwords and words found later in the text lowercased.
    """
    #listen idk why it won't just let me put in an optional repeat either
    all_words = []
    
    #three words
    all_words.extend([match for match in re.findall('[A-Z]\S* [A-Z]\S* [A-Z]\S+', txt) if 
                      all([(removePunct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(removePunct(word).lower() not in txt) for word in match.split()])])
    
    #two words
    two_words = [match for match in re.findall('[A-Z]\S* [A-Z]\S+', txt) if 
                      all([(removePunct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(removePunct(word).lower() not in txt) for word in match.split()])
                      and all([match not in x for x in all_words])]
    
    all_words.extend(two_words)
    
    #one word
    one_words = [match for match in re.findall('[A-Z]\S+', txt) if 
                      (removePunct(match).lower() not in stopword_list) 
                      and (removePunct(match).lower() not in txt)
                      and all([match not in x for x in all_words])]
    
    all_words.extend(one_words)
    
    return [word for word in [' '.join([removePunct(y) for y in x.split() if removePunct(y) not in titles]) 
                      for x in all_words] if (len(word)>0)]

In [227]:
titles

['Doctor', 'Dr', 'Mr', 'Mrs', 'Ms', 'Miss', 'Msgr', 'Monsignor', 'Rev', 'Reverend', 'Hon', 'Honorable', 'Honourable', 'Prof', 'Professor', 'Madame', 'Madam', 'Lady', 'Lord', 'Sir', 'Dame', 'Master', 'Mistress', 'Chancellor', 'Principal', 'President', 'Pres', 'Warden', 'Dean', 'Regent', 'Rector', 'Provost', 'Director']

In [228]:
getCapitalizedWords(mytext)

['American Commonwealths Boston', 'Lucien Carr', 'Cambridge Long', 'Peabody Museuml', 'Ldclen Carr', 'Itoughto Sftfltn', 'Carr', 'Missouris Governor', 'Missouri Irons', 'Carr', 'Missouri', 'States', 'Union', 'Union', 'Civil', 'Indeed', 'Harvard', 'French', 'Spanish', 'Missouri', 'State', 'Southern', 'Confederate', 'Mason', 'Dixons', 'Northern', 'Southerners', 'Southern', 'Missourians', 'States', 'States', 'State', 'Civil', 'Negro', 'Taken', 'States', 'Like']

In [202]:
from nltk.metrics import edit_distance

In [203]:
edit_distance('voynich','voynicch')

1

In [240]:
def getSetOfNames(txt):
    """
    Returns sets of names matched by similarity.
    """
    all_name_sets = []
    
    known_names = getNamesFollowingTitles(txt)
    potential_names = getCapitalizedWords(txt)
    
    for known in known_names:
        name_set = []
        name_set.append(known)
        name_parts = [removePunct(x).lower() for x in known.split() if removePunct(x) not in titles]
        
        try:
            author_surname_symspell._words[name_parts[-1].lower()]
            for n in potential_names:
                lookup = author_surname_symspell.lookup(n.lower(), Verbosity.CLOSEST)
                if (lookup):
                    for i, x in enumerate(lookup):
                        if (lookup[i]._term in name_parts):
                            name_set.append(n)
        
        except:
            for n in potential_names:
                for p in name_parts:
                    if (len(n)>5) and (len(p)>5):
                        if (edit_distance(n,p)<3):
                            name_set.append(n)
        
        all_name_sets.append(name_set)
        
    return all_name_sets

In [241]:
getSetOfNames(txts[0])

[['Dr. Herbert B. Adam.', 'Herbert', 'Adam'], ["Dr. Geor6 Ebers'", 'Ebers'], ['Mr. Cushing', 'Cushing'], ['Mr. Julian Hawthorne'], ['Mr. Swinburne.', 'Swinburne'], ['Dr. Malaren, Dr. Parker,', 'Parker'], ['President Andrew D. White.', 'White'], ['Mrs. Browning', 'Browning'], ['Professor E. Johnsou', 'Johnsou'], ['Dr. William'], ['Mr. Charles Dudley Warner'], ["Mr. Vedder's", 'Vedders'], ['Dr. Parker,', 'Parker'], ['Mr. Brander Matthews,'], ['Professor Thorold Rogers'], ['Dr. '], ['Mr. Julius Chambers,'], ['Dr. Joseph Parktercontinueshiseipositionuin', 'Joseph', 'Parktercontinueshiseipositionuin']]

In [162]:
'L' in title_list

True

In [164]:
name_partsx = [x for x in known_names[0].split() if removePunct(x) not in titles]

In [168]:
author_surname_symspell._words[name_partsx[-1].lower()]

1

In [199]:
lookup = author_surname_symspell.lookup('Carrm'.lower(), Verbosity.CLOSEST)

In [200]:
if (lookup):
    print('x')

x


In [201]:
for i, x in enumerate(lookup):
    print(lookup[i]._term)

carr
carry


In [187]:
for i, x in enumerate(author_surname_symspell.lookup('carm', Verbosity.CLOSEST)):
    print(author_surname_symspell.lookup('carm', Verbosity.CLOSEST)[i]._term)

carr
cram
calm
cary
car


In [86]:
txts[1]

'                                                                                                THE GADFLY.*                 DEARTH of good novels has been felt this year, so with unalloyed pleas-                 tire wve recognize in Mr. E. L. Voynich a new writer of exceptional ability, and in TYe Gaffy an intensely interesting and dramatic novel. Although not equal to Quo Vadfis, it is yet the most powerful story which has come to us since Sienkiewicz\'s remarkable book.                 The scene of The         is laid in Italy, modern Italy, the Italy of Garibaldi, and there is much in the experiences of The Gadfly which recalls Garibaldi\'s own ex- citing career. The fierce struggle between the eager young group of revolutionists and the ecclesiastical or conservative party- this is the leading motif of the book. Blut the reader s interest centers chiefly in two strong and original character s-The Gad- fly and the Padre Monticello.                 When we are first introduced to 

In [232]:
known_names = getNamesFollowingTitles(txts[1])
potential_names = getCapitalizedWords(txts[1])

In [233]:
known_names

['Mr. E. L. Voynich']

In [234]:
potential_names

['TYe Gaffy', 'Quo Vadfis', 'Padre Monticello', 'Padre Monticello', 'Henry Holt', 'South America', 'Tue Gadfly', 'GADFLY', 'DEARTH', 'Voynich', 'Although', 'Sienkiewiczs', 'Italy', 'Italy', 'Italy', 'Garibaldi', 'Garibaldis', 'Blut', 'Gad-', 'Arthur', 'Monticello', 'Arthur', 'Gemnia-be-', 'God', 'Leaving', 'Gadfly', 'VWynich', 'Padre', 'Arthur', 'Thirteen', 'Italy', 'Monticello', 'Frightful', 'Arthur', 'Gadfly', 'Arthur', 'Gadflys', 'Revolutionary', 'Gemma', 'God', 'Gemmas', 'Voyniich']

In [235]:
[x for x in potential_names if x in known_names[0]]

['Voynich']